In [1]:
from sklearn.metrics import f1_score
import os
import time
import os.path as osp
from tqdm import tqdm
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

from torchvision import datasets
from torchvision import transforms
import torchvision

import matplotlib.pyplot as plt
from PIL import Image

In [2]:
# random seed
SEED = 1
NUM_CLASS = 10

# Training
BATCH_SIZE = 128
NUM_EPOCHS = 60
EVAL_INTERVAL=1
SAVE_DIR = './log'

# Optimizer
LEARNING_RATE = 1e-3
MOMENTUM = 0.9
STEP=5
GAMMA=0.5

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(16)
        self.relu1 = nn.ReLU()

        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        self.relu2 = nn.ReLU()

        self.pool = nn.MaxPool2d(2, 2)

        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(64)
        self.relu3 = nn.ReLU()

        self.fc1 = nn.Linear(64 * 4 * 4, 128)
        self.dropout1 = nn.Dropout(0.5)
        self.relu4 = nn.ReLU()

        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(self.relu1(self.bn1(self.conv1(x))))
        x = self.pool(self.relu2(self.bn2(self.conv2(x))))
        x = self.pool(self.relu3(self.bn3(self.conv3(x))))

        x = x.view(-1, 64 * 4 * 4)

        x = self.relu4(self.dropout1(self.fc1(x)))
        x = self.fc2(x)
        return x

model = ConvNet()
model.to(device)

ConvNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU()
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu2): ReLU()
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu3): ReLU()
  (fc1): Linear(in_features=1024, out_features=128, bias=True)
  (dropout1): Dropout(p=0.5, inplace=False)
  (relu4): ReLU()
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)

In [4]:
from torch.optim import lr_scheduler

# 定义warm-up学习率调度器
def warmup_lr_scheduler(optimizer, warmup_epochs, multiplier):
    def lr_lambda(epoch):
        if epoch < warmup_epochs:
            return (multiplier - 1.0) * epoch / warmup_epochs + 1.0
        return 1.0

    return lr_scheduler.LambdaLR(optimizer, lr_lambda)

# 在训练代码中引入warm-up学习率调度器
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM)
warmup_epochs = NUM_EPOCHS//5
multiplier = 10
scheduler = warmup_lr_scheduler(optimizer, warmup_epochs, multiplier)

In [5]:
transform_cifar10_train = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_cifar10_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
transform_cifar100_train = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761)),
])

transform_cifar100_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761)),
])
transform_fashion_mnist = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])
transform_mnist = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize((0.1307, 0.1307, 0.1307), (0.3081, 0.3081, 0.3081)),
])


# train_set = torchvision.datasets.CIFAR100(root='../data', train=True,
#                                           download=True, transform=transform_cifar100_train)
# test_set = torchvision.datasets.CIFAR100(root='../data', train=False,
#                                          download=True, transform=transform_cifar100_test)



# train_set = torchvision.datasets.CIFAR10(root='../data', train=True,
#                                         download=True, transform=transform_cifar10_train)


# test_set = torchvision.datasets.CIFAR10(root='../data', train=False,
#                                        download=True, transform=transform_cifar10_test)



# train_set = torchvision.datasets.FashionMNIST(root='../data', train=True,
#                                               download=True, transform=transform_fashion_mnist)
# test_set = torchvision.datasets.FashionMNIST(root='../data', train=False,
#                                              download=True, transform=transform_fashion_mnist)


# train_set = torchvision.datasets.MNIST(root='../data', train=True,
#                                        download=True, transform=transform_mnist)
# test_set = torchvision.datasets.MNIST(root='../data', train=False,
#                                       download=True, transform=transform_mnist)



In [6]:
def load_dataset(dataset_name):
    if dataset_name == 'CIFAR10':
        transform_train, transform_test = transform_cifar10_train, transform_cifar10_test
        Dataset = torchvision.datasets.CIFAR10
    elif dataset_name == 'CIFAR100':
        transform_train, transform_test = transform_cifar100_train, transform_cifar100_test
        Dataset = torchvision.datasets.CIFAR100
    elif dataset_name == 'FashionMNIST':
        transform_train, transform_test = transform_fashion_mnist, transform_fashion_mnist
        Dataset = torchvision.datasets.FashionMNIST
    elif dataset_name == 'MNIST':
        transform_train, transform_test = transform_mnist, transform_mnist
        Dataset = torchvision.datasets.MNIST
    else:
        raise ValueError("Unknown dataset")

    train_set = Dataset(root='../data', train=True, download=True, transform=transform_train)
    test_set = Dataset(root='../data', train=False, download=True, transform=transform_test)
    return train_set, test_set

# 使用函数
train_set, test_set = load_dataset('CIFAR10')  # 替换 'CIFAR10' 为其他数据集名称
train_dataloader = torch.utils.data.DataLoader(train_set, batch_size=BATCH_SIZE,
                                          shuffle=True, num_workers=2)
test_dataloader = torch.utils.data.DataLoader(test_set, batch_size=BATCH_SIZE,
                                         shuffle=False, num_workers=2)

class_names = train_set.classes
print(class_names)

100%|██████████| 170498071/170498071 [00:12<00:00, 13133860.33it/s]


Extracting ../data/cifar-10-python.tar.gz to ../data
Files already downloaded and verified
['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


In [7]:
def train_batch(model, image, target):
  
    output = model(image)
    loss = criterion(output, target)

    return output, loss

criterion = nn.CrossEntropyLoss()


def test_batch(model, image, target):

    output = model(image)
    loss = criterion(output, target)

    return output, loss

In [8]:

# training_loss = []
# training_acc = []
# testing_loss = []
# testing_acc = []
# start_time = time.time()  # 记录开始时间
# all_preds = []
# all_targets = []
# # 日志文件
# log_file = 'training_log.txt'
# for epoch in tqdm(range(NUM_EPOCHS)):
#     model.train()
#     torch.cuda.empty_cache()

#     ##########################
#     ### Training
#     ##########################

#     running_cls_loss = 0.0
#     running_cls_corrects = 0

#     with tqdm(enumerate(train_dataloader), total=len(train_dataloader)) as t:
#         for batch_idx, (image, target) in t:

#             image = image.to(device)
#             target = target.to(device)
#             #######################
#             # 为使用L1loss function：
#             # target_one_hot = one_hot_encoding(target, 10)  # 转换成 one-hot 编码
    
#             # train model
#             # outputs, loss = train_batch(model, image, target_one_hot)  # 使用 one-hot 编码后的目标
    
#             outputs, loss = train_batch(model, image, target)
#             #########################
#             _, preds = torch.max(outputs, 1)
    
#             loss_data = loss.data.item()
#             if np.isnan(loss_data):
#                 raise ValueError('loss is nan while training')
#             running_cls_loss += loss.item()
#             running_cls_corrects += torch.sum(preds == target.data)
    
#             loss.backward()
#             optimizer.step()
#             optimizer.zero_grad()

#     epoch_loss = running_cls_loss / len(train_set)
#     epoch_acc = running_cls_corrects.double() / len(train_set)

#     print(f'Epoch: {epoch + 1}/{NUM_EPOCHS} Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

#     #########
#     all_preds.extend(preds.cpu().numpy())
#     all_targets.extend(target.cpu().numpy())
#     f1 = f1_score(all_targets, all_preds, average='weighted')
#     print(f"f1 score in {epoch}th epoch is {f1}")

#     ############
#     training_loss.append(epoch_loss)
#     training_acc.append(epoch_acc.cpu().detach().numpy())

#     # change learning rate
#     scheduler.step()

#     ##########################
#     ### Testing
#     ##########################
#     # # eval model during training or in the last epoch

#     if (epoch + 1) % EVAL_INTERVAL == 0 or (epoch + 1) == NUM_EPOCHS:
#         print('Begin test......')
#         model.eval()

#         val_loss = 0.0
#         val_corrects = 0

#         with tqdm(enumerate(test_dataloader), total=len(test_dataloader)) as t:
#             for batch_idx, (image, target) in t:
#                 image = image.to(device)
#                 target = target.to(device)
    
#                 # test model
#                 ##################
#                 #  target_one_hot = one_hot_encoding(target, 10)  # 转换成 one-hot 编码
#                 outputs, loss = test_batch(model, image, target)
#                 #  outputs, loss = test_batch(model, image, target_one_hot)  # 使用 one-hot 编码后的目标
#                 #######################
#                 _, preds = torch.max(outputs, 1)
    
#                 val_loss += loss.item()
#                 val_corrects += torch.sum(preds == target.data)

#         val_loss = val_loss / len(test_set)
#         val_acc = val_corrects.double() / len(test_set)
#         print(f'Test Loss: {val_loss:.4f} Acc: {val_acc:.4f}')
#         testing_loss.append(val_loss)
#         testing_acc.append(val_acc.cpu().detach().numpy())
#     with open(log_file, 'a') as f:
#         f.write(f'Epoch {epoch + 1}/{NUM_EPOCHS}\n')
#         f.write(f'Train Accuracy: {epoch_acc:.4f}%\n')
#         f.write(f'Test Accuracy: {val_acc:.4f}%\n')
#         f.write('\n')

#         # save the model in last epoch
#         if (epoch + 1) == NUM_EPOCHS:

#             state = {
#                 'state_dict': model.state_dict(),
#                 'acc': epoch_acc,
#                 'epoch': (epoch + 1),
#             }

#             # check the dir
#             if not os.path.exists(SAVE_DIR):
#                 os.makedirs(SAVE_DIR)

#             # save the state
#             torch.save(state, osp.join(SAVE_DIR, 'checkpoint_%s.pth' % (str(epoch + 1))))

# end_time = time.time()  # 记录结束时间
# duration = end_time - start_time  # 计算训练时间
# print(f"训练模型用时：{duration}秒")



In [9]:
# 
# inputs, classes = next(iter(test_dataloader))
# input = inputs[0]

# ##################### Write your answer here ##################
# # input: image, model
# # outputs: predict_label, probabilities
# # predict_label is the index (or label) of the class with the highest probability from the probabilities.
# ###############################################################
# input_tensor = input.unsqueeze(0).to(device)
# model.eval()
# with torch.no_grad():
#     output = model(input_tensor)
#     probabilities = F.softmax(output, dim=1)
#     predict_label = torch.argmax(probabilities)

# predicted_class = class_names[predict_label.item()]
# predicted_probability = probabilities[0][predict_label.item()]





# # image = input.numpy().transpose((1, 2, 0))
# # plt.imshow(image)
# # plt.text(17, 30, f'Predicted Class: {predicted_class}\nProbability: {predicted_probability:.2f}',
# #             color='white', backgroundcolor='black', fontsize=8)
# # plt.show()

# # Print probabilities for each class
# print('Print probabilities for each class:')
# for i in range(len(class_names)):
#     print(f'{class_names[i]}: {probabilities[0][i].item():.4f}')

在共形预测中，覆盖率（Coverage Rate） 和 平均集大小（Average Set Size） 是两个重要的评估指标：

覆盖率：衡量预测集合中正确包含真实标签的频率。高覆盖率表明预测集合通常包含真实的标签。

平均集大小：表示平均每次预测时预测集合中包含的标签数量。较小的平均集大小通常表示预测具有较高的确定性。

这些指标帮助评估共形预测模型的效果，特别是在不确定性和可靠性方面。

In [10]:
!pip install torchcp
from torchcp.classification.scores import THR, APS, SAPS, RAPS
from torchcp.classification.predictors import SplitPredictor, ClusterPredictor, ClassWisePredictor


from torch.utils.data import DataLoader
import torch
import torch.nn as nn

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 890.0 kB/s eta 0:00:00a 0:00:01


In [11]:
# #!pip install torchcp

# log_name="CIFAR10"
# model=torch.load("CIFAR10model")
# results = []
# alphas = [0.05, 0.1, 0.15, 0.2]
# saps_weights = [0.5, 1, 1.5, 2]
# raps_penalties = [0.05, 0.1, 0.15, 0.2]
# raps_kregs = [0, 1, 2, 5]




# alpha=0.1
# weight=1
# penalty=0.1
# kreg=0

# all_predict_sets_list=[]
# for alpha in alphas:
# #     for weight in saps_weights:
# #         for penalty in raps_penalties:
# #             for kreg in raps_kregs:
#                 score_functions = [THR(), APS(), SAPS(weight=weight), RAPS(penalty=penalty, kreg=kreg)]
#                 predictors = [SplitPredictor, ClusterPredictor, ClassWisePredictor]

#                 for score_function in score_functions:
#                     for Predictor in predictors:
#                         predictor = Predictor(score_function=score_function, model=model)
#                         predictor.calibrate(train_dataloader, alpha=alpha)

#                         all_predict_sets = []
#                         for batch in test_dataloader:
#                             images, _ = batch
#                             images = images.to(device)
#                             predict_set = predictor.predict(images)
#                             all_predict_sets.extend(predict_set)
#                         all_predict_sets_list.append(all_predict_sets)
#                         result_dict = predictor.evaluate(test_dataloader)

#                         result_str = (f"Dataset: {type(train_set).__name__}, "
#                                       f"Score Function: {score_function.__class__.__name__}, "
#                                       f"Predictor: {Predictor.__name__}, "
#                                       f"Alpha: {alpha}, Weight: {weight if isinstance(score_function, SAPS) else 'N/A'}, "
#                                       f"Penalty: {penalty if isinstance(score_function, RAPS) else 'N/A'}, "
#                                       f"kreg: {kreg if isinstance(score_function, RAPS) else 'N/A'}, "
#                                       f"Coverage Rate: {result_dict['Coverage_rate']}, "
#                                       f"Average Size: {result_dict['Average_size']}")
#                         results.append(result_str)

# # 打印或处理存储的结果
# with open(f'12.27result_{log_name}.txt', 'a') as file:
#     file.write("alpha loop"+"\n")
# for result in results:
#     #print(result)
#     with open(f'12.27result_{log_name}.txt', 'a') as file:
#         file.write(result+"\n")
# for all_predict_set in all_predict_sets_list:
#     #print(all_predict_set)
#     with open(f'12.27result_{log_name}.txt', 'a') as file:
#         file.write(' '.join(all_predict_set)+"\n")
    
    



# alpha=0.1
# weight=1
# penalty=0.1
# kreg=0
# results = []
# all_predict_sets_list=[]
# # for alpha in alphas:
# #     for weight in saps_weights:
# #         for penalty in raps_penalties:
# for kreg in raps_kregs:
#                 score_functions = [THR(), APS(), SAPS(weight=weight), RAPS(penalty=penalty, kreg=kreg)]
#                 predictors = [SplitPredictor, ClusterPredictor, ClassWisePredictor]

#                 for score_function in score_functions:
#                     for Predictor in predictors:
#                         predictor = Predictor(score_function=score_function, model=model)
#                         predictor.calibrate(train_dataloader, alpha=alpha)

#                         all_predict_sets = []
#                         for batch in test_dataloader:
#                             images, _ = batch
#                             images = images.to(device)
#                             predict_set = predictor.predict(images)
#                             all_predict_sets.extend(predict_set)
#                         all_predict_sets_list.append(all_predict_sets)
#                         result_dict = predictor.evaluate(test_dataloader)

#                         result_str = (f"Dataset: {type(train_set).__name__}, "
#                                       f"Score Function: {score_function.__class__.__name__}, "
#                                       f"Predictor: {Predictor.__name__}, "
#                                       f"Alpha: {alpha}, Weight: {weight if isinstance(score_function, SAPS) else 'N/A'}, "
#                                       f"Penalty: {penalty if isinstance(score_function, RAPS) else 'N/A'}, "
#                                       f"kreg: {kreg if isinstance(score_function, RAPS) else 'N/A'}, "
#                                       f"Coverage Rate: {result_dict['Coverage_rate']}, "
#                                       f"Average Size: {result_dict['Average_size']}")
#                         results.append(result_str)

# # 打印或处理存储的结果
# with open(f'12.27result_{log_name}.txt', 'a') as file:
#     file.write("kreg loop"+"\n")
# for result in results:
#     #print(result)
#     with open(f'12.27result_{log_name}.txt', 'a') as file:
#         file.write(result+"\n")
# for all_predict_set in all_predict_sets_list:
#     #print(all_predict_set)
#     with open(f'12.27result_{log_name}.txt', 'a') as file:
#         file.write(' '.join(all_predict_set)+"\n")


# alpha=0.1
# weight=1
# penalty=0.1
# kreg=0
# results = []
# all_predict_sets_list=[]
# # for alpha in alphas:
# for weight in saps_weights:
# #         for penalty in raps_penalties:
# #             for kreg in raps_kregs:
#                 score_functions = [THR(), APS(), SAPS(weight=weight), RAPS(penalty=penalty, kreg=kreg)]
#                 predictors = [SplitPredictor, ClusterPredictor, ClassWisePredictor]

#                 for score_function in score_functions:
#                     for Predictor in predictors:
#                         predictor = Predictor(score_function=score_function, model=model)
#                         predictor.calibrate(train_dataloader, alpha=alpha)

#                         all_predict_sets = []
#                         for batch in test_dataloader:
#                             images, _ = batch
#                             images = images.to(device)
#                             predict_set = predictor.predict(images)
#                             all_predict_sets.extend(predict_set)
#                         all_predict_sets_list.append(all_predict_sets)
#                         result_dict = predictor.evaluate(test_dataloader)

#                         result_str = (f"Dataset: {type(train_set).__name__}, "
#                                       f"Score Function: {score_function.__class__.__name__}, "
#                                       f"Predictor: {Predictor.__name__}, "
#                                       f"Alpha: {alpha}, Weight: {weight if isinstance(score_function, SAPS) else 'N/A'}, "
#                                       f"Penalty: {penalty if isinstance(score_function, RAPS) else 'N/A'}, "
#                                       f"kreg: {kreg if isinstance(score_function, RAPS) else 'N/A'}, "
#                                       f"Coverage Rate: {result_dict['Coverage_rate']}, "
#                                       f"Average Size: {result_dict['Average_size']}")
#                         results.append(result_str)

# # 打印或处理存储的结果
# with open(f'12.27result_{log_name}.txt', 'a') as file:
#     file.write("weight loop"+"\n")
# for result in results:
#     #print(result)
#     with open(f'12.27result_{log_name}.txt', 'a') as file:
#         file.write(result+"\n")
# for all_predict_set in all_predict_sets_list:
#     #print(all_predict_set)
#     with open(f'12.27result_{log_name}.txt', 'a') as file:
#         file.write(' '.join(all_predict_set)+"\n")
        
        

# alpha=0.1
# weight=1
# penalty=0.1
# kreg=0
# results = []
# all_predict_sets_list=[]
# # for alpha in alphas:
# #     for weight in saps_weights:
# for penalty in raps_penalties:
# #             for kreg in raps_kregs:
#                 score_functions = [THR(), APS(), SAPS(weight=weight), RAPS(penalty=penalty, kreg=kreg)]
#                 predictors = [SplitPredictor, ClusterPredictor, ClassWisePredictor]

#                 for score_function in score_functions:
#                     for Predictor in predictors:
#                         predictor = Predictor(score_function=score_function, model=model)
#                         predictor.calibrate(train_dataloader, alpha=alpha)

#                         all_predict_sets = []
#                         for batch in test_dataloader:
#                             images, _ = batch
#                             images = images.to(device)
#                             predict_set = predictor.predict(images)
#                             all_predict_sets.extend(predict_set)
#                         all_predict_sets_list.append(all_predict_sets)
#                         result_dict = predictor.evaluate(test_dataloader)

#                         result_str = (f"Dataset: {type(train_set).__name__}, "
#                                       f"Score Function: {score_function.__class__.__name__}, "
#                                       f"Predictor: {Predictor.__name__}, "
#                                       f"Alpha: {alpha}, Weight: {weight if isinstance(score_function, SAPS) else 'N/A'}, "
#                                       f"Penalty: {penalty if isinstance(score_function, RAPS) else 'N/A'}, "
#                                       f"kreg: {kreg if isinstance(score_function, RAPS) else 'N/A'}, "
#                                       f"Coverage Rate: {result_dict['Coverage_rate']}, "
#                                       f"Average Size: {result_dict['Average_size']}")
#                         results.append(result_str)

# # 打印或处理存储的结果
# with open(f'12.27result_{log_name}.txt', 'a') as file:
#     file.write("penalty loop"+"\n")
# for result in results:
#     #print(result)
#     with open(f'12.27result_{log_name}.txt', 'a') as file:
#         file.write(result+"\n")
# for all_predict_set in all_predict_sets_list:
#     #print(all_predict_set)
#     with open(f'12.27result_{log_name}.txt', 'a') as file:
#         file.write(' '.join(all_predict_set)+"\n")


In [12]:

import torch.nn as nn
def evaluate_and_log_results(model, alphas, weights, penalties, kregs, train_dataloader, test_dataloader, device, log_name):
    results = []
    all_predict_sets_list = []

    for alpha in alphas:
        for weight in weights:
            for penalty in penalties:
                for kreg in kregs:
                    score_functions = [THR(), APS(), SAPS(weight=weight), RAPS(penalty=penalty, kreg=kreg)]
                    predictors = [SplitPredictor, ClusterPredictor, ClassWisePredictor]

                    for score_function in score_functions:
                        for Predictor in predictors:
                            predictor = Predictor(score_function=score_function, model=model)
                            predictor.calibrate(train_dataloader, alpha=alpha)

                            all_predict_sets = []
                            for batch in test_dataloader:
                                images, _ = batch
                                images = images.to(device)
                                predict_set = predictor.predict(images)
                                all_predict_sets.extend(predict_set)
                            all_predict_sets_list.append(all_predict_sets)
                            result_dict = predictor.evaluate(test_dataloader)

                            result_str = (f"Dataset: {type(train_set).__name__}, "
                                          f"Score Function: {score_function.__class__.__name__}, "
                                          f"Predictor: {Predictor.__name__}, "
                                          f"Alpha: {alpha}, Weight: {weight if isinstance(score_function, SAPS) else 'N/A'}, "
                                          f"Penalty: {penalty if isinstance(score_function, RAPS) else 'N/A'}, "
                                          f"kreg: {kreg if isinstance(score_function, RAPS) else 'N/A'}, "
                                          f"Coverage Rate: {result_dict['Coverage_rate']}, "
                                          f"Average Size: {result_dict['Average_size']}")
                            results.append(result_str)

    # Write results to file
    with open(f'{log_name}_results_new.txt', 'a') as file:
        for result in results:
            file.write("****************")
            file.write(result + "\n")
        for all_predict_set in all_predict_sets_list:
            predict_set_str = [' '.join(map(str, sublist)) for sublist in all_predict_set]
            file.write('\n'.join(predict_set_str) + "\n")



In [ ]:
alphas = [0.05, 0.1, 0.15, 0.2]
saps_weights = [0.5, 1, 1.5, 2]
raps_penalties = [0.05, 0.1, 0.15, 0.2]
raps_kregs = [0, 1, 2, 5]
log_name="CIFAR10"
model=torch.load("CIFAR10model")
evaluate_and_log_results(model, [0.1], [1], [0.1], raps_kregs, train_dataloader, test_dataloader, device, log_name)
evaluate_and_log_results(model, [0.1], [1], raps_penalties, [0], train_dataloader, test_dataloader, device, log_name)
evaluate_and_log_results(model, [0.1], saps_weights, [0.1], [0], train_dataloader, test_dataloader, device, log_name)
evaluate_and_log_results(model, alphas, [1], [0.1], [0], train_dataloader, test_dataloader, device, log_name)

In [ ]:
print("**")

In [ ]:
#print(all_predict_sets)

In [ ]:
#torch.save(model,"CIFAR10model")